In [1]:
import os
import sys
import pandas as pd

TOP_DIR=os.path.abspath("../../")
if not TOP_DIR in sys.path: sys.path.append(TOP_DIR)

In [2]:
from prepare import LMS_EXTRACT, DASHBOARD_DIR, create_table, save_files, summarise, labour_market_status_variables, long_term_unemployed_variables
from scripts.util.metadata import read_meta, extract_dates

Load in the LMS data

In [3]:
lms_extract = pd.read_csv(
    LMS_EXTRACT,
    parse_dates=['lms_period']
)

Extract relevant metrics, pivot and save file for labour market

In [4]:
labour_market_status = lms_extract.pipe(
    create_table, labour_market_status_variables
).pipe(
    save_files, 'labour_market_status'
)
labour_market_status

variable,age_16_to_17_not_in_ft_education_in_employment_sa,age_18_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_not_in_ft_education_in_employment_sa,men_16_to_24_not_in_ft_education_in_employment_sa,women_16_to_24_not_in_ft_education_in_employment_sa,age_16_to_24_in_ft_education_in_employment_sa,age_16_to_17_not_in_ft_education_unemployed_sa,age_18_to_24_not_in_ft_education_unemployed_sa,age_16_to_24_not_in_ft_education_unemployed_sa,men_16_to_24_not_in_ft_education_unemployed_sa,...,men_16_to_24_not_in_ft_education_total_sa,women_16_to_24_not_in_ft_education_total_sa,unemployment_rate_sa,economic_inactivity_rate_sa,age_16_to_24_in_employment_sa,age_16_to_24_unemployed_sa,age_16_to_24_economically_inactive_sa,age_16_to_24_unemployment_rate_sa,quarter_label,quarter_axis_label
lms_period,,,,,,,,,,,,,,,,,,,,,
2000-03-01,246.0,2753.0,2999.0,1632.0,1368.0,935.0,85.0,329.0,415.0,251.0,...,2027.0,1952.0,5.8,23.2,3934.0,582.0,1746.0,12.9,Feb-Apr 2000,Feb-Apr\n2000
2000-06-01,241.0,2731.0,2972.0,1615.0,1357.0,958.0,75.0,318.0,393.0,234.0,...,2004.0,1917.0,5.4,23.3,3931.0,530.0,1806.0,11.9,May-Jul 2000,May-Jul\n2000
2000-09-01,234.0,2754.0,2988.0,1619.0,1369.0,937.0,84.0,322.0,406.0,234.0,...,2016.0,1944.0,5.5,23.3,3924.0,562.0,1808.0,12.5,Aug-Oct 2000,Aug-Oct\n2000
2000-12-01,244.0,2752.0,2996.0,1637.0,1359.0,951.0,79.0,312.0,391.0,238.0,...,2025.0,1927.0,5.2,23.3,3947.0,540.0,1836.0,12.0,Nov-Jan 2001,Nov-Jan\n2001
2001-03-01,249.0,2769.0,3019.0,1637.0,1382.0,937.0,77.0,324.0,401.0,244.0,...,2043.0,1959.0,5.1,23.4,3955.0,531.0,1867.0,11.8,Feb-Apr 2001,Feb-Apr\n2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,84.0,2695.0,2779.0,1445.0,1334.0,935.0,29.0,316.0,345.0,230.0,...,2071.0,1796.0,4.2,22.1,3714.0,517.0,3015.0,12.2,Nov-Jan 2024,Nov-Jan\n2024
2024-03-01,83.0,2721.0,2804.0,1463.0,1340.0,912.0,31.0,358.0,389.0,259.0,...,2102.0,1854.0,4.4,22.2,3715.0,583.0,2982.0,13.6,Feb-Apr 2024,Feb-Apr\n2024
2024-06-01,83.0,2674.0,2757.0,1460.0,1296.0,949.0,30.0,382.0,412.0,263.0,...,2124.0,1848.0,4.3,21.9,3705.0,618.0,2991.0,14.3,May-Jul 2024,May-Jul\n2024


Extract relevant metrics for long-term unemployment, pivot and save file

In [5]:
def construct_16_24_stats(data):
    data['age_16_to_24_unemployed_sa'] = (data.age_18_to_24_unemployed_sa + data.age_16_to_17_unemployed_sa).round(0)
    data['age_16_to_24_unemployed_6_to_12_months_sa'] = (data.age_18_to_24_unemployed_6_to_12_months_sa + data.age_16_to_17_unemployed_6_to_12_months_sa).round(0)
    data['age_16_to_24_unemployed_over_12_months_sa'] = (data.age_18_to_24_unemployed_over_12_months_sa + data.age_16_to_17_unemployed_over_12_months_sa).round(0)
    data['age_16_to_24_unemployed_6_to_12_months_rate_sa'] = (data['age_16_to_24_unemployed_6_to_12_months_sa'] / \
      data['age_16_to_24_unemployed_sa'] * 100).round(1)
    data['age_16_to_24_unemployed_over_12_months_rate_sa'] = (data['age_16_to_24_unemployed_over_12_months_sa'] / \
      data['age_16_to_24_unemployed_sa'] * 100).round(1)

    # Handle missing data by using the 18-24 figure instead
    data['missing_over_12_months_data'] = data.age_16_to_24_unemployed_over_12_months_sa.isna()
    data.age_16_to_24_unemployed_over_12_months_sa = data.age_16_to_24_unemployed_over_12_months_sa.mask(
        data.missing_over_12_months_data,
        data.age_18_to_24_unemployed_over_12_months_sa
    )
    data.age_16_to_24_unemployed_over_12_months_rate_sa = data.age_16_to_24_unemployed_over_12_months_rate_sa.mask(
        data.missing_over_12_months_data,
        data.age_18_to_24_unemployed_over_12_months_rate_sa.round(1)
    )
    return data

def add_axis_labels(data):
    data['quarter_axis_label'] = data.quarter_label.str.replace(' ', '\\n')
    return data    

In [6]:
long_term_unemployed = lms_extract.pipe(create_table, long_term_unemployed_variables).pipe(construct_16_24_stats).pipe(add_axis_labels)
long_term_unemployed.pipe(save_files, 'long_term_unemployed')

variable,age_16_to_17_unemployed_sa,age_18_to_24_unemployed_sa,age_16_to_17_unemployed_6_to_12_months_sa,age_16_to_17_unemployed_over_12_months_sa,age_16_to_17_unemployed_over_12_months_rate_sa,age_18_to_24_unemployed_6_to_12_months_sa,age_18_to_24_unemployed_over_12_months_sa,age_18_to_24_unemployed_over_12_months_rate_sa,quarter_label,quarter_axis_label,age_16_to_24_unemployed_sa,age_16_to_24_unemployed_6_to_12_months_sa,age_16_to_24_unemployed_over_12_months_sa,age_16_to_24_unemployed_6_to_12_months_rate_sa,age_16_to_24_unemployed_over_12_months_rate_sa,missing_over_12_months_data
lms_period,,,,,,,,,,,,,,,,
2000-03-01,185.0,397.0,26.0,10.0,5.6,56.0,59.0,14.9,Feb-Apr 2000,Feb-Apr\n2000,582.0,82.0,69.0,14.1,11.9,False
2000-06-01,153.0,377.0,24.0,10.0,6.4,55.0,67.0,17.7,May-Jul 2000,May-Jul\n2000,530.0,79.0,77.0,14.9,14.5,False
2000-09-01,176.0,386.0,21.0,12.0,6.7,56.0,58.0,14.9,Aug-Oct 2000,Aug-Oct\n2000,562.0,77.0,70.0,13.7,12.5,False
2000-12-01,160.0,381.0,21.0,10.0,6.0,61.0,48.0,12.5,Nov-Jan 2001,Nov-Jan\n2001,541.0,82.0,58.0,15.2,10.7,False
2001-03-01,147.0,384.0,14.0,9.0,5.9,55.0,56.0,14.5,Feb-Apr 2001,Feb-Apr\n2001,531.0,69.0,65.0,13.0,12.2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,120.0,397.0,12.0,6.0,5.0,67.0,73.0,18.3,Nov-Jan 2024,Nov-Jan\n2024,517.0,79.0,79.0,15.3,15.3,False
2024-03-01,131.0,452.0,20.0,14.0,11.0,78.0,86.0,19.1,Feb-Apr 2024,Feb-Apr\n2024,583.0,98.0,100.0,16.8,17.2,False
2024-06-01,104.0,515.0,14.0,18.0,17.0,61.0,86.0,16.6,May-Jul 2024,May-Jul\n2024,619.0,75.0,104.0,12.1,16.8,False


Create a summary file

In [7]:
summary = pd.concat([
    summarise(
      long_term_unemployed=long_term_unemployed,
      labour_market_status=labour_market_status,
    ),
    read_meta().pipe(extract_dates, 'LMS'),
])

summary.to_json(os.path.join(DASHBOARD_DIR, 'latest.json'), indent=2, date_format='iso')

In [8]:
summary

unemployment_rate_all_working_age                                      4.5
economic_inactivity_rate_all_working_age                              21.5
unemployment_rate_young_people                                        14.5
economic_inactivity_rate_young_people                                 40.8
unemployment_rate_not_in_full_time_education                          13.0
economic_inactivity_rate_not_in_full_time_education                   20.8
long_term_unemployment_rate_6_to_12_months                            15.6
long_term_unemployment_rate_over_12_months                           101.0
most_recent_lms_period                                        Nov-Jan 2025
last_update                                            2025-03-20 00:00:00
next_update                                            2025-04-15 00:00:00
dtype: object